In [ ]:
from gensim import corpora
from gensim.models import LdaModel
import json
from data_analysis.OpeningLDA import OpeningLDA
from classes.QueryHandler import QueryHandler

## LDA

In [ ]:
#opening_lda = OpeningLDA(num_passes=5, print_debug=True) 

#print(len(opening_lda.texts.keys()))
#opening_lda.lda_model.print_topics(num_words=15)

#opening_lda.save('lda')


In [ ]:
opening_lda = OpeningLDA.load("lda")

In [ ]:
opening_lda.__construct_topic_doc_matrix__()

In [ ]:
query_handler = QueryHandler()
query_handler.opening_lda = opening_lda

sample_query1 = { 
                 "message": "capture the center, develop light square bishop",
                 "color": "white" 
}
query_handler.handle_user_query(sample_query1, debug=True)

## GPT

In [1]:
from data_analysis.OpeningGPT import OpeningGPT
from scraper.Paths import Paths

opening_gpt = OpeningGPT()


### Precomputing summaries for all ECOs 

In [2]:
opening_gpt.precompute_all_summaries(save_to=Paths.RAND_SUMMARIES_JSON, print_debug=True, temperature=0.8, min_sum_len=100, max_sum_len=300, overwrite_existing=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (15040 > 1024). Running this sequence through the model will result in indexing errors


OpeningGPT: precompute_all_summaries - Skipping ECO "B44" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "D12" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "B88" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "D15" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "B43" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "B81" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "B75" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "D23" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "D24" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skippin

Your max_length is set to 300, but your input_length is only 104. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)


	n = 3736
	num_chars = 3705
	len(tokenized chunk) = 1019 | 3719
	summary: "The French Defence is a chess opening characterised by the moves 1.e4 e6 and 2.d4 d5. Black usually plays ...c5 soon after, attacking White's pawn centre and gaining space on the queenside. The McCutcheon Variation is named for John Lindsay McCutheon of Philadelphia (1857–1905), who brought the variation to public attention when he used it to defeat World Champion Steinitz in a simultaneous exhibition in Manhattan in 1885. The Burn Variation, the Morozevich Variation and the Advance Variation are the most difficult to play against due to the closed structure and unique strategies of the system. Black's natural French Defence move 6...c5 runs into 7.Bxe7 when Black must either move the king with 7...Kxe7 or allow 7...Qxe7 8.Nb5!"
	summary len = 177

OpeningGPT: __process_text__ - Iteration #1
	chars_processed = 3705/4196
	toks_processed = 1019/1133
	n = 491
	num_chars = 481
	len(tokenized chunk) = 107 | 481
	summ

Your max_length is set to 300, but your input_length is only 107. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)


...

Your max_length is set to 300, but your input_length is only 163. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=81)


	n = 3714
	num_chars = 3683
	len(tokenized chunk) = 1008 | 3687
	summary: "The French Defence is named after a match played by correspondence between the cities of London and Paris in 1834. Black usually plays ...c5 soon after, attacking White's pawn centre and gaining space on the queenside. The French has a reputation for solidity and resilience, although some lines such as the Winawer Variation can lead to sharp complications. Black's position is often somewhat cramped in the early game; in particular, the pawn on e6 can impede the development of the bishop on c8. Despite the symmetrical pawn structure, White cannot force a draw. An obsession with obtaining one one pawn sometimes results in embarrassment for White. White will often play c2–c2 at some stage to put pressure on Black's d5-pawn. Black can give White an isolated pawn on c4 to give White's pieces greater freedom."
	summary len = 172

OpeningGPT: __process_text__ - Iteration #1
	chars_processed = 3683/4332
	toks_processed 

Your max_length is set to 300, but your input_length is only 80. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)


	n = 3244
	num_chars = 3219
	len(tokenized chunk) = 1013 | 3228
	summary: "1.e4 e5 2.Nf3 Nc6 3.Bc4 Bc5 4.Nc3 Nf6 5.Nxe4 d5 7.Neg5+ Kg8 8.d3 h6 9.Nh3 Bg4 4...d5 5.0-0 6.Bd3 7.Bxe4 Bd6 8.Nxd4 exd4 10.Qxd4 0-0 11.Be3 (Tartakower–Atkins, London 1922[5]) and 11.Qxe4 Qe7 (11.0? Bxh2+ wins) Black has a decisive advantage after 5...Bb4."
	summary len = 148

OpeningGPT: __process_text__ - Iteration #2
	chars_processed = 6460/6820
	toks_processed = 2031/2141
	n = 361
	num_chars = 280
	len(tokenized chunk) = 78 | 281
	summary: "The Halloween Gambit is one of the most popular chess moves of all time. It is a variation of the famous Three Knights Defense. It has been used in a number of chess tournaments, including the 2010 World Chess Championship in Moscow, Russia. The trick is to get the opponent to play a certain way in order to win. For more information on the game, see: http://www.chess.com/halloween-gambit/ and http:// www.cheek.org/Halloween- Gambit."
	summary len = 106

OpeningGPT: __proce

KeyboardInterrupt: 

In [ ]:
opening_gpt.randomize_all_summaries(print_debug=True)